<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1 Подготовка

In [1]:
import pandas as pd 
import numpy as np 
import re
import nltk
from tqdm import tqdm
nltk.download('wordnet')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt 
import seaborn as sns 

import catboost as cb
from catboost import CatBoostClassifier

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [46]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [47]:
df = df.drop(['Unnamed: 0'], axis =1)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [29]:
corpus = df['text'].values


In [5]:
print(f"Количество дубликатов: {df.duplicated().sum()}")

Количество дубликатов: 0


In [30]:
display(df['toxic'].value_counts())
class_ratio = df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]
class_ratio

0    143106
1     16186
Name: toxic, dtype: int64

8.841344371679229

In [31]:
wnl = WordNetLemmatizer()

def clear_and_lemm(text):
    pattern = re.sub(r'[^a-zA-Z]', ' ', text)
    clear = pattern.split()
    lemm = []
    for i in range(len(clear)):
        lemm.append(wnl.lemmatize(clear[i]))
    return " ".join(lemm)

<div class="alert alert-block alert-info">

<b>Совет:</b> Для более корректной работы WordNetLemmatizer желательно передавать, еще POS-теги слов
</div>



In [33]:
# с помощью итератора tgdm задаем цикл по исполнению функции лемматизации текста и очистки его от лишних символов 
for i in tqdm(range(len(corpus))):
    corpus[i] = clear_and_lemm(corpus[i])

100%|██████████| 159292/159292 [00:35<00:00, 4451.41it/s]


In [48]:
df_corpus = pd.DataFrame(corpus)
df['lemm_text'] = df_corpus[0]

display(df.head())
df.info()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       159292 non-null  object
 1   toxic      159292 non-null  int64 
 2   lemm_text  159292 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.6+ MB


#### Вывод по первому разделу:

- Входе подготовки данных к обучению моделей, мы получили корректные данные без пропусков и дубликатов. Была проведена лимматизация текста, так же были удалены "лишние символы", которые будут занимать дополнительный объем памяти при векторизации. 

# 2 Обучение моделей 

In [10]:
features = df['lemm_text']
target = df['toxic']

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.4, 
                                                                              random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=0.4,
                                                                            random_state=12345)


In [11]:
# избавляемся от лишних слов в дата фрейме 
try:
    nltk.download('stopwords')
except:
    pass
#Объявляю набор стоп-слов 
try:
    stopwords = set(stopwords.words('english'))
except:
    pass
#Создаем TFIDF-векторизатор
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
#Производим векторизацию текстов
features_train = count_tf_idf.fit_transform(features_train)
features_valid = count_tf_idf.transform(features_valid)
features_test = count_tf_idf.transform(features_test)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)
print(target_train.shape)
print(target_valid.shape)
print(target_test.shape)

(95575, 121115)
(38230, 121115)
(25487, 121115)
(95575,)
(38230,)
(25487,)


#### Для обучения и последующего анализа были выбраны модели Логистическая регрессия, Случайный лес и модель градиентного бустинга CatBoost.

### Логистическая регрессия 

In [13]:
#Обучаем и проверяе Логистическую регрессию на кросс-валидации, указываю параметр class_weight = 'balanced'
regression = LogisticRegression(fit_intercept=True, 
                                class_weight='balanced', 
                                random_state=42,
                                solver='liblinear'
                               )
#Определяю словарь с набором параметров
regression_parametrs = {'C': [0.1, 1, 10]}

#Применяю GridSearchCV с кросс-валидацией
regression_grid = GridSearchCV(regression, regression_parametrs, scoring='f1', cv=3)
regression_grid.fit(features_train, target_train)

regression.fit(features_train, target_train)
regression_cv_score = cross_val_score(regression,features_train, target_train,scoring='f1',cv=3).mean()
print('Среднее качество модели Логистической регрессии на кросс-валидации:', regression_cv_score)


Среднее качество модели Логистической регрессии на кросс-валидации: 0.7442873549454857


In [14]:
regression_params = regression_grid.best_params_
regression_score = regression_grid.score(features_train, target_train)
print(regression_params)
print(regression_score)

{'C': 10}
0.9240530667179389


In [15]:
regression_model = LogisticRegression(fit_intercept=True,
                                class_weight='balanced',
                                random_state=42,
                                solver='liblinear',
                                C=regression_params['C']
                               )

regression_model.fit(features_train, target_train)
regression_model_predictions = regression_model.predict(features_valid)

In [16]:
regression_predictions = regression_model.predict(features_valid)
regression_f1 = round(f1_score(target_valid, regression_predictions), 3) 
print(f'Метрика качества F1 для Линейной регресии на валидационной выборке равна {regression_f1}')

Метрика качества F1 для Линейной регресии на валидационной выборке равна 0.757


In [17]:
features_test

<25487x121115 sparse matrix of type '<class 'numpy.float64'>'
	with 681377 stored elements in Compressed Sparse Row format>

In [19]:
# Проводим тестирование лучше модели Линейная регрессия на тестовой выборке

regression_predictions_test = regression_model.predict(features_test)
regression_f1_test  = round(f1_score(target_test, regression_predictions_test), 3) 
print(f'Метрика качества F1 для Линейной регресии на тестовой выборке равна {regression_f1_test}')


Метрика качества F1 для Линейной регресии на тестовой выборке равна 0.755


### Случайный лес 

In [20]:
#Подбираем оптимальные гиперпараметры для Случайного леса на кросс-валидации, параметр class_weight = 'balanced'
forest = RandomForestClassifier(class_weight='balanced', n_jobs=-1 )
#Определяю словарь с набором параметров
forest_parametrs = { 'n_estimators': range(20, 40, 5),
                     'max_depth': range(4, 8, 2),
                     'min_samples_leaf': range(3,5),
                     'min_samples_split': range(2,6,2)}

#Применяю GridSearchCV с кросс-валидацией
forest_grid = GridSearchCV(forest, forest_parametrs, scoring='f1', cv=3)
forest_grid.fit(features_train, target_train)

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              n_jobs=-1),
             param_grid={'max_depth': range(4, 8, 2),
                         'min_samples_leaf': range(3, 5),
                         'min_samples_split': range(2, 6, 2),
                         'n_estimators': range(20, 40, 5)},
             scoring='f1')

In [21]:
forest_params = forest_grid.best_params_
forest_score = forest_grid.score(features_train, target_train)
print(forest_params)
print(forest_score)

{'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 25}
0.30103085178969885


In [22]:
forest_model = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced',
                                     max_depth=forest_params['max_depth'],
                                     min_samples_leaf = forest_params['min_samples_leaf'],
                                     min_samples_split = forest_params['min_samples_split'],
                                     n_estimators = forest_params['n_estimators'])

forest_model.fit(features_train, target_train)
forest_model_predictions = forest_model.predict(features_valid)

In [23]:
forest_predictions = forest_model.predict(features_valid)
forest_f1 =  round(f1_score(target_valid, forest_predictions), 3)
print(f'Метрика качества F1 для Случайного леса на валидационной выборке равна {forest_f1}')

Метрика качества F1 для Случайного леса на валидационной выборке равна 0.297


### Модель Градиентного бустинга - CatBoost

In [24]:
features_train_cb = features_train[:10000]
features_valid_cb = features_valid[:3000]
target_train_cb = target_train[:10000]
target_valid_cb = target_valid[:3000]
features_test_cb = features_test[:1500]
target_test_cb = target_test[:1500]

In [25]:
#Объявляем модель с учетом дисбаланса классов
cb = cb.CatBoostClassifier(class_weights=[0, 1], iterations=30)
#Определяю словарь с набором параметров
cb_parametrs = {'depth': [4, 8]}

cb_grid = GridSearchCV(cb, cb_parametrs, scoring='f1', cv=3)
cb_grid.fit(features_train_cb, target_train_cb, verbose=10)

Learning rate set to 0.5
0:	learn: 0.0192028	total: 153ms	remaining: 4.43s
10:	learn: 0.0003448	total: 1s	remaining: 1.74s
20:	learn: 0.0003442	total: 1.83s	remaining: 786ms
29:	learn: 0.0003436	total: 2.65s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.0140209	total: 129ms	remaining: 3.74s
10:	learn: 0.0003514	total: 1000ms	remaining: 1.73s
20:	learn: 0.0003514	total: 1.83s	remaining: 785ms
29:	learn: 0.0003514	total: 2.59s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.0128264	total: 105ms	remaining: 3.06s
10:	learn: 0.0002281	total: 941ms	remaining: 1.63s
20:	learn: 0.0002280	total: 1.76s	remaining: 757ms
29:	learn: 0.0002288	total: 2.53s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.0221050	total: 946ms	remaining: 27.4s
10:	learn: 0.0003674	total: 7.29s	remaining: 12.6s
20:	learn: 0.0003252	total: 13.7s	remaining: 5.87s
29:	learn: 0.0003250	total: 19.9s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.0180815	total: 976ms	remaining: 28.3s
10:	learn: 0.0003109	to

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x7f110ad359a0>,
             param_grid={'depth': [4, 8]}, scoring='f1')

In [26]:
cb_params = cb_grid.best_params_
cb_score = cb_grid.score(features_train_cb, target_train_cb)
print(cb_params)
print(cb_score)

{'depth': 8}
0.17618092285245304


In [27]:
import catboost as cb
from catboost import CatBoostClassifier
cb_model = cb.CatBoostClassifier(class_weights=[0, 1], depth=cb_params['depth'], iterations=50)
cb_model.fit(features_train_cb, target_train_cb, verbose=10)
cb_model_predictions = cb_model.predict(features_valid_cb)

cb_f1 = round(f1_score(target_valid_cb, cb_model_predictions), 3)
print(cb_f1)

Learning rate set to 0.429515
0:	learn: 0.0220754	total: 1.27s	remaining: 1m 2s
10:	learn: 0.0002449	total: 10.1s	remaining: 35.7s
20:	learn: 0.0002441	total: 19s	remaining: 26.3s
30:	learn: 0.0002437	total: 28.2s	remaining: 17.3s
40:	learn: 0.0002437	total: 37.1s	remaining: 8.15s
49:	learn: 0.0002437	total: 45.2s	remaining: 0us
0.185


In [28]:
columns = ['Модель', 'F1-метрика', 'F1- метрика на тесте']

regression_model = ['Логистическая регрессия',  regression_f1, regression_f1_test]
forest_model = ['Случайный лес',  forest_f1, 'не проводилось']
cd_model = ['CatBoostRegressor',  cb_f1, 'не проводилось']

table = pd.DataFrame([regression_model, forest_model, cd_model], columns = columns)


display(table)

,Модель,F1-метрика,F1- метрика на тесте
0,Логистическая регрессия,0.757,0.755
1,Случайный лес,0.297,не проводилось
2,CatBoostRegressor,0.185,не проводилось


# 3 Выводы: 

- На этапе подготовки данных была проведена лемматизация комментариев пользователей и удаление лишних символов из текстов. 


- Для анализа данных и построения модели предоставлен датасет с размеченными данными, содержащий комментарии пользователей к товарам, доступным для приобретения в интернет-магазине «Викишоп». Датасет состоит из 2 столбцов с данными и 159292 строк. 


- В качестве моделей использованы Логистическая регрессия, Случайный лес и CatBoost классификатор. 
По результатам подбора гиперпараметров и обучения моделей наилучший результат метрики F1 показала Логистическа регерссия, CatBoost и Случайный лес, выглядят значительно хуже, что выглядит немного странно. 


- Возможно стоит подключить к сравнительному анализу модель BERT - тогда картина о качестве предсказаний будет более ясной, поэтому на данном этапе и в данных условиях можно заключить, что Линейная лучше остальных справляется с бинарной классификацией, но вопрос является ли она лучшим алгоритмом для решения подобных задач с текстами остается открытым   

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны